Celda1

In [1]:
!pip install flask flask_sqlalchemy pyngrok python-dotenv requests-oauthlib


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Celda 2

In [2]:
# Sustituye el token por el tuyo personal
token = "2vPtl6kHq0BkIZiRGWDzGl3kizc_2MyFkWmTdpMgN5Je8QQEP"

with open(".env", "w") as f:
    f.write(f"NGROK_AUTHTOKEN={token}\n")

print("✅ .env creado con éxito")


✅ .env creado con éxito


Celda 3

In [3]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///messungen.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.secret_key = 'clave_secreta'

db = SQLAlchemy(app)

class Usuario(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(150))
    correo = db.Column(db.String(150), unique=True)
    contraseña = db.Column(db.String(150))
    origen = db.Column(db.String(50))  # google o manual

with app.app_context():
    db.create_all()
    print("📦 Base de datos creada")


📦 Base de datos creada


Celda 4

In [4]:
from pyngrok import ngrok, conf
from dotenv import load_dotenv
import os

load_dotenv()
ngrok_token = os.getenv("NGROK_AUTHTOKEN")
conf.get_default().auth_token = ngrok_token

public_url = ngrok.connect(5000)
print("🌐 Tu app está disponible en:", public_url)

🌐 Tu app está disponible en: NgrokTunnel: "https://0751-186-27-213-35.ngrok-free.app" -> "http://localhost:5000"


Celda 5

In [5]:
from requests_oauthlib import OAuth2Session

# Reemplaza por tus valores reales
CLIENT_ID = "944897940833-kkushpcet8k9semtced5oulapb86np9a.apps.googleusercontent.com"
CLIENT_SECRET = "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"
REDIRECT_URI = str(public_url) + "/google/callback"

AUTHORIZATION_BASE_URL = "https://accounts.google.com/o/oauth2/auth"
TOKEN_URL = "https://accounts.google.com/o/oauth2/token"
SCOPE = ["https://www.googleapis.com/auth/userinfo.email", "https://www.googleapis.com/auth/userinfo.profile"]

print("🔗 URI de redirección autorizada que debes usar en Google Cloud Console:")
print(REDIRECT_URI)


🔗 URI de redirección autorizada que debes usar en Google Cloud Console:
NgrokTunnel: "https://0751-186-27-213-35.ngrok-free.app" -> "http://localhost:5000"/google/callback


Celda 6

In [8]:
from flask import render_template_string, request, redirect, session, url_for
import requests
# Esto limpia rutas anteriores para que no dé error si vuelves a correr la celda
app.view_functions.clear()
@app.route('/')
def inicio():
    return render_template_string("""
    <h2>Bienvenido a <strong>Messungen</strong></h2>
    <a href='/login/google'><button>🔐 Iniciar sesión con Google</button></a><br><br>
    <a href='/login'><button>📧 Iniciar sesión manual</button></a><br><br>
    <a href='/register'><button>📝 Crear cuenta manual</button></a>
    """)

@app.route('/login/google')
def login_google():
    google = OAuth2Session(CLIENT_ID, scope=SCOPE, redirect_uri=REDIRECT_URI)
    authorization_url, state = google.authorization_url(AUTHORIZATION_BASE_URL, access_type="offline", prompt="select_account")
    session['oauth_state'] = state
    return redirect(authorization_url)

@app.route('/google/callback')
def callback_google():
    google = OAuth2Session(CLIENT_ID, state=session['oauth_state'], redirect_uri=REDIRECT_URI)
    token = google.fetch_token(TOKEN_URL, client_secret=CLIENT_SECRET, authorization_response=request.url)

    r = google.get("https://www.googleapis.com/oauth2/v1/userinfo").json()
    correo = r['email']
    nombre = r.get('name', 'Usuario Google')

    usuario = Usuario.query.filter_by(correo=correo).first()
    if not usuario:
        usuario = Usuario(nombre=nombre, correo=correo, contraseña="", origen="google")
        db.session.add(usuario)
        db.session.commit()

    session['usuario'] = usuario.correo
    return redirect('/dashboard')

@app.route('/dashboard')
def dashboard():
    if 'usuario' not in session:
        return redirect('/')
    return f"Hola {session['usuario']} 👋 Bienvenido a tu panel."

@app.route('/logout')
def logout():
    session.clear()
    return redirect('/')


Celda 7

In [9]:
from werkzeug.security import generate_password_hash, check_password_hash

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        nombre = request.form['nombre']
        correo = request.form['correo']
        contraseña = request.form['contraseña']
        if Usuario.query.filter_by(correo=correo).first():
            return "❌ El correo ya está registrado."
        usuario = Usuario(nombre=nombre, correo=correo, contraseña=generate_password_hash(contraseña), origen="manual")
        db.session.add(usuario)
        db.session.commit()
        session['usuario'] = correo
        return redirect('/dashboard')
    return render_template_string("""
    <h3>Registro Manual</h3>
    <form method='POST'>
        <input name='nombre' placeholder='Nombre'><br><br>
        <input name='correo' placeholder='Correo'><br><br>
        <input name='contraseña' type='password' placeholder='Contraseña'><br><br>
        <button type='submit'>Crear cuenta</button>
    </form>
    """)

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        correo = request.form['correo']
        contraseña = request.form['contraseña']
        usuario = Usuario.query.filter_by(correo=correo).first()
        if usuario and check_password_hash(usuario.contraseña, contraseña):
            session['usuario'] = usuario.correo
            return redirect('/dashboard')
        return "❌ Datos inválidos"
    return render_template_string("""
    <h3>Iniciar sesión Manual</h3>
    <form method='POST'>
        <input name='correo' placeholder='Correo'><br><br>
        <input name='contraseña' type='password' placeholder='Contraseña'><br><br>
        <button type='submit'>Entrar</button>
    </form>
    """)


Celda 8

In [ ]:
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Apr/2025 20:03:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:03:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2025 20:03:59] "GET /login/google HTTP/1.1" 302 -
127.0.0.1 - - [09/Apr/2025 20:04:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:04:30] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:04:36] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:04:53] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [09/Apr/2025 20:04:53] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:05:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 20:05:12] "GET /login/google HTTP/1.1" 302 -
